## Create Edge List
Creates an edge list for each patch in a provided patch raster and cost surface

In [32]:
#Import libraries
import arcpy
from arcpy.sa import *
import numpy as np
import pandas as pd

#Set workspaces
arcpy.env.workspace = 'C:/Workspace/PronghornConnectivity/PronghornConnectivity.gdb'
arcpy.env.scratchWorkspace = 'C:/Workspace/PronghornConnectivity/Scratch/Scratch.gdb'
arcpy.env.overwriteOutput = True

In [17]:
def msg(txt):
    print(txt)
    arcpy.AddMessage(txt)
    return

In [84]:
#Get input datasets: Patches and CostSurface
patchRaster = 'C:/Workspace/PronghornConnectivity/PronghornConnectivity.gdb/PatchCores'
if not arcpy.Exists(patchRaster):
    print("Cannot locate patch raster")

costSurface = 'C:/Workspace/PronghornConnectivity/PronghornConnectivity.gdb/CostSurface'
if not arcpy.Exists(costSurface):
    print("Cannot locate cost surface")
    
maxCostDistance = 100000

In [73]:
#Get list of patch IDs
#Create empty list
patchIDs = []
#Convert values to numpy array
arr = arcpy.da.TableToNumPyArray(patchRaster,"Value")
#Loop through records
for i in arr.tolist():
    patchIDs.append(i[0])

In [88]:
#Get the source patch
for patchID in patchIDs[:5]:
    msg("Running patch {}".format(patchID))

    #Get the source patch
    msg("Isolating source patch...")
    thePatch = Con(patchRaster,patchID,"","VALUE = {}".format(patchID))

    #Calculate cost distance from the patch
    msg("Computing cost distance...")
    theCostDist = CostDistance(thePatch,costSurface,maxCostDistance)

    #Computing LCP distance to other Patches
    msg("Computing LCP distances to other patches")
    outZTable = "zmin{}".format(patchID)
    zMin = ZonalStatisticsAsTable(patchRaster,"Value",theCostDist,outZTable,"DATA","Minimum")

    ##Convert table to a pandas dataframe
    msg("Creating table")
    #Convert arcTable to numpy Array
    arrZS = arcpy.da.TableToNumPyArray(outZTable,['VALUE','MIN'])
    #Convert to pandas dataframe
    dfZS = pd.DataFrame(arrZS)
    #Rename columns
    dfZS.columns = ['To_ID','Cost']
    #Add the from ID column
    dfZS['From_ID'] = patchID
    #Rearrange columns
    dfZS = dfZS[["From_ID","To_ID","Cost"]]

    #If first table, make the output df; otherwise append records
    if patchID == 1:
        dfOut = dfZS.copy(deep=True)
    else:
        msg("Adding records")
        dfOut = pd.concat([dfOut,dfZS[dfZS.To_ID > dfZS.From_ID]])

#Write the list to csv file
dfOut.to_csv("EdgeList.csv",index=False)